In [1]:
import boto3
import json, uuid, copy, datetime
import random, names, tabulate
import pandas as pd

In [ ]:
# open_cmd_in_new_terminal("docker-compose up")
!gnome-terminal -- docker-compose up

In [ ]:
# !./setuprds.sh
!./setupdynamo.sh

In [ ]:
!./deploylambdas.sh

In [ ]:
dynamodb = boto3.client('dynamodb', endpoint_url = 'http://localhost:4569')
local_lambda = boto3.client('lambda', endpoint_url = 'http://localhost:4574')

In [2]:
# Use this one for staging
dynamodb = boto3.client('dynamodb', region_name = 'us-east-1')
local_lambda = boto3.client('lambda', region_name = 'us-east-1')

In [ ]:
# Setup the RDS roles
role_payload = { "type": "CREATE_ROLES", "credentials": {
        "account_api_worker": "pwd_for_account_api",
        "save_tx_api_worker": "pwd_for_transaction_api",
        "float_api_worker": "pwd_for_float_api",
        "auth_api_worker": "pwd_for_auth_api_worker"
    }
}
local_lambda.invoke(FunctionName='db-migration-local-migrate', InvocationType='RequestResponse', Payload=json.dumps(role_payload))

In [9]:
# Setup the RDS tables
local_lambda.invoke(FunctionName='db-migration-local-migrate', InvocationType='RequestResponse', Payload=json.dumps({'type': 'SETUP_TABLES'}))

{'ResponseMetadata': {'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/json',
   'content-length': '68',
   'access-control-allow-origin': '*',
   'server': 'Werkzeug/0.15.4 Python/3.6.8',
   'date': 'Tue, 09 Jul 2019 09:33:59 GMT'},
  'RetryAttempts': 0},
 'StatusCode': 200,
 'Payload': <botocore.response.StreamingBody at 0x7ff96f8f8fd0>}

In [3]:
def print_deployed_functions():
    function_list = local_lambda.list_functions()
    # print(function_list)
    print('Function list: ', [function['FunctionName'] for function in function_list['Functions']])

In [11]:
def generate_account(client_id = 'zar_client_co', float_id = 'zar_cash_float'):
    first_name = names.get_first_name()
    family_name = names.get_last_name()
    user_id = str(uuid.uuid4())
    return { 'clientId': client_id, 'defaultFloatId': float_id, 'ownerUserId': user_id, 'userFirstName': first_name, 'userFamilyName': family_name}

In [12]:
def decode_lambda_result(lambda_result):
    lambda_payload = lambda_result['Payload'].read()
    lambda_pload_decoded = lambda_payload.decode('utf-8')
    lambda_pload_object = json.loads(lambda_pload_decoded)
    return lambda_pload_object

In [5]:
def create_number_accounts(number_accounts = 1, client_id = 'zar_client_co', lambda_name = 'account-create'):
    account_dicts = [generate_account(client_id) for i in range(number_accounts)]
    persisted_accounts = []
    for account in account_dicts:
        lambda_result = local_lambda.invoke(FunctionName='user_existence_api', InvocationType='RequestResponse', 
                                           Payload=json.dumps(account))
        lambda_payload = json.loads(decode_lambda_result(lambda_result)['body'])
        persisted_account = copy.deepcopy(account)
        persisted_account['accountId'] = lambda_payload['accountId']
        persisted_account['persistedTime'] = lambda_payload['persistedTime']
        print('Account with ID %s persisted at %s' % (persisted_account['accountId'], persisted_account['persistedTime']))
        persisted_accounts.append(persisted_account)
    
    return persisted_accounts
    

In [ ]:
def generate_saving_transaction(account_id, ref_amount = 100, client_id = 'zar_client_co', float_id = 'zar_cash_primary'):
    current_time = datetime.datetime.now()
    saved_amount = round(random.random() * ref_amount * 10 * 10) # random proportion of ref amount, 
    test_saving_dict = { 
        'accountId': account_id, 
        'initiationTime': str(current_time), 
        'settlementTime': str(current_time), 
        'savedAmount': saved_amount, 
        'savedCurrency': 'ZAR', 
        'savedUnit': 'HUNDREDTH_CENT',
        'floatId': float_id,
        'clientId': client_id
    }
    return test_saving_dict

In [ ]:
def seed_savings_for_accounts(accounts, tx_per_account = 1, base_amount = 100):
    transactions = []
    for i in range(tx_per_account):
        transactions.extend([generate_saving_transaction(account['accountId'], base_amount) for account in accounts])
#     print('transactions: ', transactions)
    tx_records = []
    for tx in transactions:
        lambda_result = local_lambda.invoke(FunctionName='activity-save', InvocationType='RequestResponse', 
                                           Payload=json.dumps(tx, default=str))
        lambda_payload = json.loads(decode_lambda_result(lambda_result)['body'])
        persisted_tx = copy.deepcopy(tx)
        persisted_tx['transactionId'] = lambda_payload['transactionDetails'][0][0]['transaction_id']
        persisted_tx['currentBalance'] = lambda_payload['newBalance']['amount']
        # persisted_account['persistedTime'] = lambda_payload['persistedTime']
        print('Tx for account %s persisted with new balance %s' % (tx['accountId'], persisted_tx['currentBalance']))
        tx_records.append(persisted_tx)
    
    return tx_records

In [ ]:
def run_accrual_for_float(client_id = 'zar_client_co', float_id = 'zar_cash_primary', accrued_amount = 100):
    accrualDict = {
        'clientId': client_id,
        'floatId': float_id,
        'accrualAmount': accrued_amount,
        'currency': 'ZAR',
        'unit': 'HUNDREDTH_CENT',
        'backingEntityIdentifier': 'tx-id-backing'
    }
    lambda_result = local_lambda.invoke(FunctionName='float-accrue', InvocationType='RequestResponse', 
                                               Payload=json.dumps(accrualDict, default=str))
    lambda_payload = json.loads(decode_lambda_result(lambda_result)['body'])
    return lambda_payload

In [4]:
print_deployed_functions()

Function list:  ['user_existence_api', 'insert_user_credentials', 'user_activity_api', 'verify_jwt', 'sign_jwt', 'db_migration', 'db_migration_auth', 'float_api', 'update_password', 'seed_initial_admin', 'verify_user_credentials']


In [12]:
accounts = create_number_accounts(number_accounts = 1)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
account_info = [{ 'Name': account['userFirstName'] + ' ' + account['userFamilyName'], 
                'AccountId': account['accountId'] } for account in accounts[:100]]
print(tabulate.tabulate(account_info, headers = 'keys'))

In [ ]:
single_tx = generate_saving_transaction(accounts[0]['accountId'], 100)
lambda_result = local_lambda.invoke(FunctionName='activity-save', Payload=json.dumps(single_tx, default=str))
result_decoded = decode_lambda_result(lambda_result)
print('Lambda result', result_decoded)

In [ ]:
account_balances = seed_savings_for_accounts(accounts[:100], tx_per_account = 3, base_amount = 100 * 100 * 100)

In [ ]:
def merge_dataframes(accounts, account_balances):
    df = pd.DataFrame(account_balances)
    df = df[df['currentBalance'] == df.groupby('accountId')['currentBalance'].transform('max')]
    df = df[['accountId', 'currentBalance']]
    df['currentBalance'] = pd.to_numeric(df['currentBalance'])
    df['balanceRand'] = df['currentBalance']/10000
    account_name_df = pd.DataFrame(accounts)[['accountId', 'floatId', 'userFamilyName', 'userFirstName']]
    account_name_df['Holder Name'] = account_name_df['userFirstName'] + ' ' + account_name_df['userFamilyName']
    account_name_df.set_index('accountId')
    df.set_index('accountId')
    merged_df = pd.merge(account_name_df, df)[['accountId', 'floatId', 'Holder Name', 'balanceRand']]
    return merged_df

In [ ]:
merged_df = merge_dataframes(accounts, account_balances)

In [ ]:
merged_df

In [ ]:
amount_to_accrue = round(random.random() * 100 * 100 * 100)
print('Instructing accrual of : R', amount_to_accrue / (100 * 100))
accrual_result = run_accrual_for_float(accrued_amount = amount_to_accrue)
allocation_details = json.loads(accrual_result['userAllocationTransactions']['body'])
account_transaction_list = allocation_details['allocationRecords']['accountTxIds']
# print(tabulate.tabulate(account_transaction_list, headers = 'keys'))

In [ ]:
# print(accrual_result)
account_transactions = allocation_details['allocationRecords']['accountTxIds']
number_of_float_transactions = len(allocation_details['allocationRecords']['floatTxIds'])
number_of_account_transactions = len(account_transactions)
print('%d float transactions, %d account transactions' % (number_of_float_transactions, number_of_account_transactions))
sum_of_accounts = sum([transaction['amount'] for transaction in account_transactions])
print('Sum of account additions: ', sum_of_accounts)
bonus_allocation = accrual_result['entityAllocations']['bonusShare']
company_allocation = accrual_result['entityAllocations']['clientShare']
print('Bonus allocation: %d, and client co allocation: %d' % (bonus_allocation, company_allocation))
total_allocations = sum_of_accounts + bonus_allocation + company_allocation
print('Of accrual %d, allocations %d' % (amount_to_accrue, total_allocations))
rounding_allocated_to_bonus = isinstance(allocation_details['bonusAllocation'], list)
print('Was there a rounding allocation? : ', rounding_allocated_to_bonus)
print('Rounding allocation record: ', allocation_details['bonusAllocation'])

In [ ]:
balance_payload = { 
    'accountId': accounts[0]['accountId'], 
    'clientId': 'zar_client_co', 
    'floatId': 'zar_cash_primary',
    'currency': 'ZAR',
    'timezone': 'Africa/Johannesburg'
}
balance_projection = local_lambda.invoke(FunctionName='activity-balance', Payload=json.dumps(balance_payload))
projection_decoded = decode_lambda_result(balance_projection)
print(projection_decoded)

In [ ]:
def fetch_account_balance(accountId, clientId = 'zar_client_co', floatId = 'zar_cash_primary', currency = 'ZAR', timezone = 'Africa/Johannesburg'):
    balance_payload = { 
        'accountId': accountId, 
        'clientId': 'zar_client_co', 
        'floatId': 'zar_cash_primary',
        'currency': 'ZAR',
        'timezone': 'Africa/Johannesburg'
    }
    balance_projection = local_lambda.invoke(FunctionName='activity-balance', Payload=json.dumps(balance_payload))
    projection_decoded = decode_lambda_result(balance_projection)
    return projection_decoded

In [ ]:
def fetch_account_balances(accounts):
#     balance_dicts = [fetch_account_balance(account['accountId']) for account in accounts]
    balance_dicts = []
    for account in accounts:
#         print('account: ', account)
        balance_dict = fetch_account_balance(account['accountId'])
        balance_dict['accountId'] = account['accountId']
        projected_dict = {
            'accountId': balance_dict['accountId'],
            'currentBalanceZAR': balance_dict['currentBalance']['amount'] / 10000,
            'balanceEndOfDayZAR': balance_dict['balanceEndOfToday']['amount'] / 10000,
            'balanceNextDayZAR': balance_dict['balanceSubsequentDays'][0]['amount'] / 10000
        }
        balance_dicts.append(projected_dict)
    
    return balance_dicts

In [ ]:
balances_projections = fetch_account_balances(accounts)

In [ ]:
balance_df = pd.DataFrame(balances_projections)
balance_df = balance_df[['accountId', 'currentBalanceZAR', 'balanceEndOfDayZAR', 'balanceNextDayZAR']]
balance_df